# AML Task 2 : Disease Classification

Useful links :
* [combining models in Keras](https://machinelearningmastery.com/model-averaging-ensemble-for-deep-learning-neural-networks/#:~:text=The%20simplest%20way%20to%20develop,each%20of%20the%20trained%20models.)

* [technique to handle imbalanced data](https://www.kdnuggets.com/2017/06/7-techniques-handle-imbalanced-data.html)

* [TensorFlow MultiClass Classification with MLP](https://medium.com/@canerkilinc/hands-on-tensorflow-2-0-multi-label-classifications-with-mlp-88fc97d6a7e6)

## Install

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow
import sys
!{sys.executable} -m pip install pandas

## Imports

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.utils import shuffle
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from random import randrange

print('Imports : done')

DEBUG:matplotlib:$HOME=/home/atroska
DEBUG:matplotlib:CONFIGDIR=/home/atroska/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.0.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'op

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:matplotlib:CACHEDIR=/home/atroska/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/atroska/.cache/matplotlib/fontlist-v300.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


Imports : done


## Read data

In [3]:
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
sample=pd.read_csv("sample.csv")

In [4]:
X_train_copy=X_train.to_numpy()[:,1:]
y_train_copy=y_train.to_numpy()[:,1:]
X_test_copy=X_test.to_numpy()[:,1:]

In [5]:
print('Data read and copy')

Data read and copy


In [ ]:
y_train.describe()

In [ ]:
X_test.describe()

In [ ]:
y_train_copy

## Pre-processing DATA

### Under-sampling 

In [ ]:
#Take randomly 600 sample from the sample with classification 
indexes = np.where(y_train_copy==1)[0]
count = len(indexes)
for i in range(3000):
    random = randrange(count)
    index = indexes[random]
    X_train_copy = np.delete(X_train_copy, index, 0)
    y_train_copy = np.delete(y_train_copy, index, 0)
    indexes = np.where(y_train_copy==1)[0]
    count = len(indexes)
print('Under-sampling done')

In [ ]:
len(X_train_copy)==len(y_train_copy)

In [ ]:
len(X_train_copy)

## Ensemble different resampled datasets

In [4]:
def create6balancedsets(x, y):
    
    #TO BE CONTINUED
    x_0 = x[np.where(y == 0)[0]]
    y_0 = y[np.where(y == 0)[0]]
    x_2 = x[np.where(y == 2)[0]]
    y_2 = y[np.where(y == 2)[0]]
    
    indexes = np.where(y==1)[0]
    count = len(indexes)
    
    x1s = []
    y1s = []
    
    for i in range(6):
        xi = []
        yi = []
        
        for j in range(600):
            random = randrange(count)
            index = indexes[random]
        
            xi.append(x[index,:])
            yi.append(y[index,:])
            indexes = np.where(y==1)[0]
            count = len(indexes)
        
        x1s.append(xi)
        y1s.append(yi)
    
    print(x1s)
    print(y1s)
    print('Creat 6 balanced set done')

In [ ]:
create6balancedsets(X_train_copy, y_train_copy)

In [6]:
def resampledDataSet(X_train_copy, y_train_copy):
    
    #Get all the sample on class 0 and 2
    features_0=X_train_copy[np.where(y_train_copy==0)[0]]
    features_2=X_train_copy[np.where(y_train_copy==2)[0]]
    
    #Save their classification / just recreated them would have work too
    y_0=y_train_copy[np.where(y_train_copy==0)[0]]
    y_2=y_train_copy[np.where(y_train_copy==2)[0]]
    
    #Do the same thing for classification 1
    X_label1=X_train_copy[np.where(y_train_copy==1)[0]]
    y_1abel1=y_train_copy[np.where(y_train_copy==1)[0]]
    X_label1, y_1abel1= shuffle(X_label1, y_1abel1, random_state=0)

    #augmented/resample datasets
    #With classification 0
    X_train_augmented=tf.concat([X_train_copy,features_0 ,features_0,features_0,features_0],axis=0) 
    y_train_augmented=tf.concat([y_train_copy.flatten(),tf.zeros(4*features_0.shape[0],dtype="int64")],axis=0)
    
    #With classification 1
    X_train_augmented=tf.concat([X_train_augmented,features_2,features_2,features_2,features_2],axis=0) #along the rows
    y_train_augmented=tf.concat([y_train_augmented,2*tf.ones(4*features_2.shape[0],dtype="int64")],0)

    # Shuffle the data to not biased the model after
    X_train_augmented=tf.random.shuffle(X_train_augmented,seed=8).numpy()
    y_train_augmented=tf.random.shuffle(y_train_augmented,seed=8).numpy().reshape(y_train_augmented.shape[0],1)
  
    #Getting the datasets chunks of label 1
    label1_chunks=[]
    chunks_size=int(X_label1.shape[0]/6)
    for i in range(6):
      label1_chunks.append((X_label1[chunks_size*i:chunks_size*(i+1)],y_1abel1[chunks_size*i:chunks_size*(i+1)]))
  
    #preparing training sets for models
    model_sets=[(X_train_augmented,y_train_augmented)]
    for i in range(6):
      model_set=((np.concatenate((label1_chunks[i][0],features_0,features_2),axis=0),np.concatenate((label1_chunks[i][1],y_0,y_2),axis=0)
   ))
      model_sets.append(shuffle(model_set[0],model_set[1],random_state=1))
    
    return model_sets

In [7]:
sets = resampledDataSet(X_train_copy, y_train_copy)

In [8]:
sets

[(array([[-4.63722289e-01,  7.82043993e-01,  5.81221282e-01, ...,
           1.45173538e+00,  2.01866507e+00,  3.74492466e-01],
         [-4.61253300e-02, -7.55765200e-01,  1.06449440e-01, ...,
           6.73705459e-01,  1.05565190e+00,  3.02166820e-01],
         [-2.65768000e-04,  1.74704447e-01,  8.36888671e-01, ...,
           1.15566886e+00,  7.22084522e-01,  5.68624318e-01],
         ...,
         [-5.97559214e-01, -1.45493582e-01,  8.12333047e-01, ...,
           5.05009234e-01, -1.58621266e-01,  2.18159080e+00],
         [-2.01162115e-01, -7.07909465e-01,  6.51777506e-01, ...,
           3.92371565e-01,  8.39085996e-01,  6.82330060e-02],
         [-6.24226034e-01,  8.96051645e-01,  5.99530518e-01, ...,
           6.38048232e-01,  1.29018784e+00, -3.32590200e-03]]), array([[1],
         [2],
         [0],
         ...,
         [1],
         [0],
         [0]])),
 [array([[-0.08463734, -0.90885973,  0.78172427, ..., -0.43757296,
           2.41921592,  1.73236382],
         [-1.

## Model configuration

Using the tutorial https://medium.com/@canerkilinc/hands-on-tensorflow-2-0-multi-label-classifications-with-mlp-88fc97d6a7e6

In [9]:
# Model 1 : score -> 0.64
# Model 1, under samling randomly -> 0.68
def mymodel1(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(250, activation=tf.nn.relu),
        tf.keras.layers.Dense(125, activation=tf.nn.relu),
        tf.keras.layers.Dense(62, activation=tf.nn.relu),
        tf.keras.layers.Dense(31, activation=tf.nn.relu),
        tf.keras.layers.Dense(15, activation=tf.nn.relu),
        tf.keras.layers.Dense(7, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="adam",
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [10]:
# Model 2 : flatten and droupout layers, score -> 0.61
def mymodel2(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(250, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(125, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(62, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(31, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(15, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(7, activation=tf.nn.relu),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="adam",
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [11]:
# Model 3
def mymodel3(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1000, activation=tf.nn.tanh),
        tf.keras.layers.Dense(500, activation=tf.nn.tanh),
        tf.keras.layers.Dense(250, activation=tf.nn.tanh),
        tf.keras.layers.Dense(125, activation=tf.nn.tanh),
        tf.keras.layers.Dense(62, activation=tf.nn.tanh),
        tf.keras.layers.Dense(31, activation=tf.nn.tanh),
        tf.keras.layers.Dense(15, activation=tf.nn.tanh),
        tf.keras.layers.Dense(7, activation=tf.nn.tanh),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="Adadelta" ,
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [12]:
# Model 4
def mymodel4(x, y, batchsize=32, epochs=50):
    #Configure the model (with dropout)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1000, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(500, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(250, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(125, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(62, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(31, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(15, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(7, activation=tf.nn.tanh),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(3, activation="softmax")
    ])
    
    #Compile the model
    model.compile(optimizer="Adadelta" ,
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics=["sparse_categorical_accuracy"])
    
    #Fit the model
    model.fit(x, y, batch_size=batchsize, epochs=epochs)

    #return the model
    return model

In [13]:
def multiModel(sets, epochs):
    models = []
    batchs = [16,32]
    for simple_set in sets:
        print('train 4 neural networks')
        for epoch in epochs:
            for  batch in batchs:
                models.append(mymodel1(simple_set[0], simple_set[1], batch, epoch))
                models.append(mymodel2(simple_set[0], simple_set[1], batch, epoch))
                models.append(mymodel3(simple_set[0], simple_set[1], batch, epoch))
                models.append(mymodel4(simple_set[0], simple_set[1], batch, epoch))
    return models

### Simple model

In [ ]:
M = mymodel1(X_train_copy, y_train_copy)

### Multi-model

In [14]:
Ms = multiModel(sets, [20,30,40,50])

Epoch 1/20
600/600 [==============================] - 4s 6ms/step - loss: 0.6968 - sparse_categorical_accuracy: 0.6965
Epoch 2/20
600/600 [==============================] - 4s 7ms/step - loss: 0.4167 - sparse_categorical_accuracy: 0.8380
Epoch 3/20
600/600 [==============================] - 4s 6ms/step - loss: 0.2133 - sparse_categorical_accuracy: 0.9232
Epoch 4/20
600/600 [==============================] - 4s 6ms/step - loss: 0.1381 - sparse_categorical_accuracy: 0.9522
Epoch 5/20
600/600 [==============================] - 4s 6ms/step - loss: 0.0973 - sparse_categorical_accuracy: 0.9728
Epoch 6/20
600/600 [==============================] - 4s 6ms/step - loss: 0.0769 - sparse_categorical_accuracy: 0.9747
Epoch 7/20
600/600 [==============================] - 4s 6ms/step - loss: 0.0525 - sparse_categorical_accuracy: 0.9822
Epoch 8/20
600/600 [==============================] - 4s 6ms/step - loss: 0.0410 - sparse_categorical_accuracy: 0.9891
Epoch 9/20
600/600 [============================

300/300 [==============================] - 2s 7ms/step - loss: 0.0281 - sparse_categorical_accuracy: 0.9909
Epoch 10/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0156 - sparse_categorical_accuracy: 0.9962
Epoch 11/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0532 - sparse_categorical_accuracy: 0.9843
Epoch 12/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0285 - sparse_categorical_accuracy: 0.9921
Epoch 13/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0373 - sparse_categorical_accuracy: 0.9889
Epoch 14/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0145 - sparse_categorical_accuracy: 0.9966
Epoch 15/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0197 - sparse_categorical_accuracy: 0.9961
Epoch 16/20
300/300 [==============================] - 2s 7ms/step - loss: 0.0359 - sparse_categorical_accuracy: 0.9893
Epoch 17/20
300/300 [==============================]

600/600 [==============================] - 4s 6ms/step - loss: 0.0362 - sparse_categorical_accuracy: 0.9918
Epoch 18/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0271 - sparse_categorical_accuracy: 0.9922
Epoch 19/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0550 - sparse_categorical_accuracy: 0.9870
Epoch 20/30
600/600 [==============================] - 4s 7ms/step - loss: 0.0065 - sparse_categorical_accuracy: 0.9979
Epoch 21/30
600/600 [==============================] - 4s 7ms/step - loss: 0.0331 - sparse_categorical_accuracy: 0.9918
Epoch 22/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0542 - sparse_categorical_accuracy: 0.9871
Epoch 23/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0059 - sparse_categorical_accuracy: 0.9983
Epoch 24/30
600/600 [==============================] - 4s 6ms/step - loss: 0.0017 - sparse_categorical_accuracy: 0.9994
Epoch 25/30
600/600 [==============================]

600/600 [==============================] - 5s 8ms/step - loss: 0.6007 - sparse_categorical_accuracy: 0.8067
Epoch 26/30
600/600 [==============================] - 4s 7ms/step - loss: 0.5945 - sparse_categorical_accuracy: 0.8106
Epoch 27/30
600/600 [==============================] - 5s 8ms/step - loss: 0.5884 - sparse_categorical_accuracy: 0.8168
Epoch 28/30
600/600 [==============================] - 5s 8ms/step - loss: 0.5826 - sparse_categorical_accuracy: 0.8198
Epoch 29/30
600/600 [==============================] - 5s 8ms/step - loss: 0.5766 - sparse_categorical_accuracy: 0.8242
Epoch 30/30
600/600 [==============================] - 5s 9ms/step - loss: 0.5708 - sparse_categorical_accuracy: 0.8280
train ### neural network#s
Epoch 1/30
300/300 [==============================] - 2s 7ms/step - loss: 0.7870 - sparse_categorical_accuracy: 0.6309
Epoch 2/30
300/300 [==============================] - 2s 7ms/step - loss: 0.4194 - sparse_categorical_accuracy: 0.8343
Epoch 3/30
300/300 [=======

300/300 [==============================] - 2s 8ms/step - loss: 0.8891 - sparse_categorical_accuracy: 0.6144
Epoch 3/30
300/300 [==============================] - 3s 9ms/step - loss: 0.8609 - sparse_categorical_accuracy: 0.6413
Epoch 4/30
300/300 [==============================] - 2s 8ms/step - loss: 0.8434 - sparse_categorical_accuracy: 0.6556
Epoch 5/30
300/300 [==============================] - 2s 8ms/step - loss: 0.8303 - sparse_categorical_accuracy: 0.6665
Epoch 6/30
300/300 [==============================] - 2s 8ms/step - loss: 0.8196 - sparse_categorical_accuracy: 0.6746
Epoch 7/30
300/300 [==============================] - 2s 8ms/step - loss: 0.8106 - sparse_categorical_accuracy: 0.6804
Epoch 8/30
300/300 [==============================] - 2s 8ms/step - loss: 0.8026 - sparse_categorical_accuracy: 0.6878
Epoch 9/30
300/300 [==============================] - 2s 8ms/step - loss: 0.7952 - sparse_categorical_accuracy: 0.6932
Epoch 10/30
300/300 [==============================] - 2s 8

...

113/113 [==============================] - 1s 6ms/step - loss: 0.2274 - sparse_categorical_accuracy: 0.9244
Epoch 10/50
113/113 [==============================] - 1s 6ms/step - loss: 0.1805 - sparse_categorical_accuracy: 0.9439
Epoch 11/50
113/113 [==============================] - 1s 6ms/step - loss: 0.1446 - sparse_categorical_accuracy: 0.9478
Epoch 12/50
113/113 [==============================] - 1s 6ms/step - loss: 0.1857 - sparse_categorical_accuracy: 0.9544
Epoch 13/50
113/113 [==============================] - 1s 6ms/step - loss: 0.0913 - sparse_categorical_accuracy: 0.9700
Epoch 14/50
113/113 [==============================] - 1s 6ms/step - loss: 0.0498 - sparse_categorical_accuracy: 0.9839
Epoch 15/50
113/113 [==============================] - 1s 6ms/step - loss: 0.0570 - sparse_categorical_accuracy: 0.9822
Epoch 16/50
113/113 [==============================] - 1s 6ms/step - loss: 0.0530 - sparse_categorical_accuracy: 0.9889
Epoch 17/50
113/113 [==============================]

113/113 [==============================] - 1s 7ms/step - loss: 0.2317 - sparse_categorical_accuracy: 0.9222
Epoch 27/50
113/113 [==============================] - 1s 8ms/step - loss: 0.2144 - sparse_categorical_accuracy: 0.9311
Epoch 28/50
113/113 [==============================] - 1s 8ms/step - loss: 0.2010 - sparse_categorical_accuracy: 0.9261
Epoch 29/50
113/113 [==============================] - 1s 7ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9456
Epoch 30/50
113/113 [==============================] - 1s 7ms/step - loss: 0.2400 - sparse_categorical_accuracy: 0.9311
Epoch 31/50
113/113 [==============================] - 1s 7ms/step - loss: 0.1770 - sparse_categorical_accuracy: 0.9478
Epoch 32/50
113/113 [==============================] - 1s 7ms/step - loss: 0.1778 - sparse_categorical_accuracy: 0.9483
Epoch 33/50
113/113 [==============================] - 1s 7ms/step - loss: 0.1825 - sparse_categorical_accuracy: 0.9456
Epoch 34/50
113/113 [==============================]

113/113 [==============================] - 1s 8ms/step - loss: 0.7042 - sparse_categorical_accuracy: 0.7194
Epoch 45/50
113/113 [==============================] - 1s 8ms/step - loss: 0.7017 - sparse_categorical_accuracy: 0.7239
Epoch 46/50
113/113 [==============================] - 1s 8ms/step - loss: 0.6993 - sparse_categorical_accuracy: 0.7239
Epoch 47/50
113/113 [==============================] - 1s 8ms/step - loss: 0.6966 - sparse_categorical_accuracy: 0.7239
Epoch 48/50
113/113 [==============================] - 1s 8ms/step - loss: 0.6944 - sparse_categorical_accuracy: 0.7278
Epoch 49/50
113/113 [==============================] - 1s 8ms/step - loss: 0.6919 - sparse_categorical_accuracy: 0.7300
Epoch 50/50
113/113 [==============================] - 1s 8ms/step - loss: 0.6895 - sparse_categorical_accuracy: 0.7350
train ### neural network#s
Epoch 1/50
57/57 [==============================] - 0s 8ms/step - loss: 0.8564 - sparse_categorical_accuracy: 0.5661
Epoch 2/50
57/57 [==========

57/57 [==============================] - 0s 8ms/step - loss: 0.5846 - sparse_categorical_accuracy: 0.7556
Epoch 12/50
57/57 [==============================] - 0s 8ms/step - loss: 0.5391 - sparse_categorical_accuracy: 0.7889
Epoch 13/50
57/57 [==============================] - 0s 8ms/step - loss: 0.4912 - sparse_categorical_accuracy: 0.8239
Epoch 14/50
57/57 [==============================] - 0s 8ms/step - loss: 0.4796 - sparse_categorical_accuracy: 0.8050
Epoch 15/50
57/57 [==============================] - 0s 8ms/step - loss: 0.4298 - sparse_categorical_accuracy: 0.8383
Epoch 16/50
57/57 [==============================] - 0s 8ms/step - loss: 0.3502 - sparse_categorical_accuracy: 0.8772
Epoch 17/50
57/57 [==============================] - 0s 8ms/step - loss: 0.3552 - sparse_categorical_accuracy: 0.8806
Epoch 18/50
57/57 [==============================] - 0s 8ms/step - loss: 0.3177 - sparse_categorical_accuracy: 0.8939
Epoch 19/50
57/57 [==============================] - 0s 8ms/step - l

57/57 [==============================] - 1s 15ms/step - loss: 0.8398 - sparse_categorical_accuracy: 0.6850
Epoch 31/50
57/57 [==============================] - 1s 13ms/step - loss: 0.8373 - sparse_categorical_accuracy: 0.6811
Epoch 32/50
57/57 [==============================] - 1s 13ms/step - loss: 0.8348 - sparse_categorical_accuracy: 0.6844
Epoch 33/50
57/57 [==============================] - 1s 12ms/step - loss: 0.8324 - sparse_categorical_accuracy: 0.6867
Epoch 34/50
57/57 [==============================] - 1s 14ms/step - loss: 0.8301 - sparse_categorical_accuracy: 0.6883
Epoch 35/50
57/57 [==============================] - 1s 10ms/step - loss: 0.8278 - sparse_categorical_accuracy: 0.6906
Epoch 36/50
57/57 [==============================] - 1s 9ms/step - loss: 0.8256 - sparse_categorical_accuracy: 0.6906
Epoch 37/50
57/57 [==============================] - 1s 9ms/step - loss: 0.8234 - sparse_categorical_accuracy: 0.6956
Epoch 38/50
57/57 [==============================] - 0s 9ms/st

## Predictions

### Simple model

In [ ]:
predictions = M.predict(X_test_copy)

In [ ]:
predictions

In [ ]:
predictions=np.argmax(predictions,axis=1)

In [ ]:
predictions

### Multi-model

In [15]:
def multimodelprediction(models, X_test):
    preds_sum = 0
    for model in models :
        preds_sum += model.predict(X_test)
    preds = np.argmax(preds_sum/len(models), axis=1)
    return preds

In [16]:
predictions = multimodelprediction(Ms, X_test_copy)

### Corss validation

In [22]:
def validation(X_train, y_train, models):
    
    preds_sum = 0
    for model in models :
        preds_sum += model.predict(X_train)
    preds = np.argmax(preds_sum/len(models), axis=1)
    
    score=balanced_accuracy_score(y_train,predictions)
    print("Score is : ")

In [23]:
score = validation(X_train_copy, y_train_copy, Ms)

ValueError: Found input variables with inconsistent numbers of samples: [4800, 4100]

## Verification

In [17]:
len(X_test_copy) ==  len(predictions)

True

## Submission

In [18]:
ids = sample.to_numpy()[:,0]
sub = {"id" : ids, "y": predictions}
sub = pd.DataFrame(sub)
print(sub.head())
sub.to_csv("submission_y_test")

    id  y
0  0.0  1
1  1.0  1
2  2.0  0
3  3.0  0
4  4.0  1


## Results

* Model 1, no pre-processing -> 0.64
* Model 1, under-sampling -> 0.68
* Model 2, no pre-processing -> 0.61
* Multi-set, resampling -> 0.724
* Multi-model & multi-set -> 0.734
* same, but 2 batch size tested -> 0.735